In [1]:
# Import the dependencies:
import pandas as pd
import sqlalchemy
import numpy as np

# Create the temporary database connection:
database_connection_string = 'sqlite:///'

# Create the database engine:
engine = sqlalchemy.create_engine(database_connection_string)

# Test the engine and database connectivity:
engine

Engine(sqlite:///)

In [2]:
# SELECTING DATA FROM MULTIPLE TABLES
# In data engineering, a visual schema often documents the relationship among the tables in a database.
# For example, a SAKILA SCHEMA. 
# This is a database schema - a logical configuaration of database tables and their relationships - for a famoust test dataset that people use to test SQL.
# The schema demonstrates how elaborate the relationships among the tables in a database can become.
# Data engineers have their work cut out for them when they have to manage tons of data.
# We won't concern ourselves with organizing and storing data in database tables in this module.
# Instead, we'll focus on how to access the data from the database to use it in our financial applications.
# Say that the dat we need to analyze spreads across multiple tables in a database.
# We need to find a way to join the table data so that it's returned as a singe, comprehensive result.
# We can do this joining by using a SQL query.
# But before we dive into joins, we'll explore how to work with and visualize joins using Pandas.

In [3]:
# VISUALIZING JOINS
# We can combine data that spreads across database tables ni various ways.
# These depend on the organization of the data in the tables, the relationships among the tables, and the purpose of the query.
# For example, suppose that a financial database contains a table of sales records.
# EAch record includes the customer identifier for that sale.
# A second table stores the customers' billing information.
# Each record in this table contains the customer identifier and other key information, such as:
    # 1. Customer name
    # 2. Address
    # 3. Credit card number
# Splitting this data into different tables was out of our control.
# Regardless, we now need information from both tables - that is, the sales records and the customer information.
# To accomplish this goal in SQL, we use JOINS via the `JOIN` clause. 
# This clause offers four main types of joins: Inner Join, Left Join, Right Join, Full Join (Also referred to as a Full Outer Join).
# Depending on our goal, we can use any of these types of joins, which represent different techniques for combining data from multiple tables:
    # 1. INNER JOIN: This selects the elements in every row that match in Table A and Table B.
    # 2. LEFT JOIN: This selects all the elements in Table A (regardless of whether they match the elements in Table B).
    # 3. RIGHT JOIN: This selects all the elements in Table B (regardless of whether they match Table A).
    # 4. FULL JOIN: This selects all the elements in Table A and in Table B (regardless of whether they match the elementsin the other table).    

In [4]:
# USING SQL TO JOIN DATA
# To explore joins in action, we'll use random data to create tables that we'll then join.
# We'll begin by generating DataFrames with random data.

# CREATE THE DATAFRAMES
# We'll use Faker, a Python package used to genreate fake data to create the following:
    # 1. A fake bank DataFrame that has bank IDs and default currencies.
    # 2. A fake customer DataFrame that has first and last names, address, bank IDs, and default bank currencies.

from faker import Faker
import random

random.seed(2)
Faker.seed(2)
fake = Faker()
available_currencies = []
bank_data = []

for i in range(10):
    currency = fake.currency_name()
    available_currencies.append(currency)
    bank_data.append({
        'bank_id': fake.swift(),
        'bank_default_currency': currency
    })
bank_dataframe = pd.DataFrame(bank_data)
bank_dataframe

,bank_id,bank_default_currency
0,CLFXGBTQ,Bahraini dinar
1,TBSVGBK1,Central African CFA franc
2,ZXQLGB826RC,Zimbabwean dollar
3,OKMNGB7K9LP,Australian dollar
4,FKFEGB66,Hungarian forint
5,OZNXGB7X,Burmese kyat
6,LOFYGBZ37P5,Venezuelan bolívar
7,QQZLGB33W93,Japanese yen
8,KWFTGBR4,Sudanese pound
9,ZWQRGB760TN,Cayman Islands dollar


In [5]:
# Notice that the DataFrame consists of two columns named 'bank_default_currency' and 'bank_id' and 10 rows of data.
# Next, we'll use Faker to generate our second DataFrame of customer data:

# NOTE:
# Don't worry about reproducing this code. In a real-world scenario, you'll work with a database that contains this information.

customer_data = []

for i in range(100):
    customer_data.append({
        'first_name': fake.first_name(),
        'last_name': fake.last_name(),
        'address': fake.address(),
        'default_currency': random.choice(available_currencies),
        'bank_id': random.choice(bank_data)['bank_id']
    })

customer_dataframe = pd.DataFrame(customer_data)
display(customer_dataframe.head(10))
display(customer_dataframe.tail(10))

,first_name,last_name,address,default_currency,bank_id
0,Tracy,Richardson,"0310 Joshua Forks\nPort Alexandraview, IA 28088",Bahraini dinar,TBSVGBK1
1,Sherry,Brown,"PSC 0055, Box 2301\nAPO AE 09343",Central African CFA franc,OZNXGB7X
2,Anthony,Daniel,22806 Laura Gateway Apt. 005\nWest Matthewtown...,Zimbabwean dollar,FKFEGB66
3,Andrea,Jacobs,"9273 Donna Island Apt. 072\nTuckermouth, FL 45203",Hungarian forint,ZWQRGB760TN
4,Jonathon,Obrien,"820 Johnson Estates\nNorth Shawn, IN 67119",Australian dollar,ZWQRGB760TN
5,Sabrina,Erickson,"71419 Claudia Union\nJonathanchester, KY 69469",Bahraini dinar,ZWQRGB760TN
6,Karen,Hernandez,"81311 Hunter Fort\nEast Andrewmouth, NH 41097",Zimbabwean dollar,LOFYGBZ37P5
7,Amanda,James,"668 Bianca Center\nHickmanshire, HI 12793",Venezuelan bolívar,KWFTGBR4
8,Elizabeth,Castillo,"545 Smith Spur\nGarciaport, ID 88656",Burmese kyat,KWFTGBR4
9,Marcia,Burnett,"3991 Justin Landing Suite 137\nNorth Ashley, M...",Japanese yen,KWFTGBR4


,first_name,last_name,address,default_currency,bank_id
90,Sierra,Robinson,"8282 Jensen Drive Apt. 820\nJessicaton, RI 98014",Zimbabwean dollar,ZXQLGB826RC
91,Charlene,Burton,"93295 Brown Valley\nPort Caitlin, ID 47872",Zimbabwean dollar,TBSVGBK1
92,Bryan,Barton,"096 Marc Pines\nNorth Jose, AS 33533",Japanese yen,OKMNGB7K9LP
93,Matthew,Le,"358 Christina Alley Apt. 666\nBrennanview, MT ...",Sudanese pound,CLFXGBTQ
94,Rachel,Smith,"533 Paige Plaza\nKellymouth, AR 15056",Australian dollar,OKMNGB7K9LP
95,Matthew,Ramsey,"4788 Carter Islands\nReynoldsfurt, CO 52286",Japanese yen,TBSVGBK1
96,Christopher,Meadows,"7916 Hayes Vista Suite 708\nGuerrastad, SC 55685",Hungarian forint,TBSVGBK1
97,Paige,Harris,"40406 Sanders Gateway\nStokesview, PA 49718",Cayman Islands dollar,OKMNGB7K9LP
98,Philip,Smith,30325 Hudson Throughway Suite 692\nNorth Curti...,Cayman Islands dollar,ZWQRGB760TN
99,Noah,Taylor,"66513 Meza Spurs Suite 674\nSouth Steven, SC 6...",Burmese kyat,FKFEGB66


In [6]:
# Notice that this DataFrame contains 100 rows of data in five  columns:
    # 1. 'address'
    # 2. 'bank_id'
    # 3. 'default_currency'
    # 4. 'first_name'
    # 5. 'last_name'

In [7]:
# CREATE THE DATABASE TABLES
# The following cod uses our DataFrames to create database tables:
bank_dataframe.to_sql('banks', engine, index=False, if_exists='replace')
customer_dataframe.to_sql('customers', engine, index=False, if_exists='replace')
engine.table_names()

C:\Users\legar\AppData\Local\Temp\ipykernel_7076\3292876666.py:5: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['banks', 'customers']

In [12]:
# JOIN THE DATABASE TABLES
# Did you notice that the tables in the original DataFrames use the bank's Society for Worlwide Interbank Financial Telecommunications (SWIFT) ID?
# The SWIFT ID is a unique identifier code for a bank.
# The IDs that `bank_dataframe` use in its `bank_id` column are the same IDs that `customer_dataframe` uses in its `bank_id` column.
# Next, we'll perform a join on the tables using the 'bank_id' column.
# The reason to choose this column is twofold: Both tables have it, and it has the common values that link the tables together.
# To join data from two tables, we need to tell SQL the first table from which to select data. 
# In this case, we'll use the `banks` table.
# Then, we need to tell SQL the second table from which to join data.
# In this case, we'll use the `customers` table. 
# Finally, we need to specify the columns on which to join the data.
# These details will tell SQL to find only the rows from both tables where the 'bank_id' columns match. 

# IMPORTANT:
# You can specify any columns on which to join tables, as long as they have data that's common in both tables.
# In this example, we could have used the 'default_currency' column in the `customers` table and the 'bank_default_currency' in the `banks` table.
# The choice of columns depends entirely on how you want to combine the data.

# The following code joins the tables on the 'bank_id' column:
query = """
SELECT *
FROM banks
JOIN customers ON banks.bank_id = customers.bank_id;
"""

joined_df = pd.read_sql_query(query, con=engine)
display(joined_df.head(10))
display(joined_df.tail(10))

,bank_id,bank_default_currency,first_name,last_name,address,default_currency,bank_id
0,CLFXGBTQ,Bahraini dinar,Aaron,Norton,"24449 Tammy Via Suite 496\nWebbfurt, WI 39109",Cayman Islands dollar,CLFXGBTQ
1,CLFXGBTQ,Bahraini dinar,Alexandria,Ramirez,Unit 5133 Box 6283\nDPO AP 51610,Bahraini dinar,CLFXGBTQ
2,CLFXGBTQ,Bahraini dinar,Andre,Fowler,"76574 Brown Road\nLake Danny, GU 12296",Central African CFA franc,CLFXGBTQ
3,CLFXGBTQ,Bahraini dinar,Brandi,Johnson,"30482 Tyler Keys\nLake Ryanside, RI 21937",Zimbabwean dollar,CLFXGBTQ
4,CLFXGBTQ,Bahraini dinar,Brenda,Marshall,"570 Tonya Walk\nCynthiaville, PR 25458",Australian dollar,CLFXGBTQ
5,CLFXGBTQ,Bahraini dinar,Curtis,Roberts,"0337 Hayes Trace\nNorth Kylehaven, RI 85697",Hungarian forint,CLFXGBTQ
6,CLFXGBTQ,Bahraini dinar,Matthew,Le,"358 Christina Alley Apt. 666\nBrennanview, MT ...",Sudanese pound,CLFXGBTQ
7,CLFXGBTQ,Bahraini dinar,Matthew,Norman,"4183 Allen Vista Suite 238\nEast Brenda, IA 52658",Australian dollar,CLFXGBTQ
8,CLFXGBTQ,Bahraini dinar,Melanie,Carrillo,"04410 Sarah Harbor\nSouth Elizabethburgh, MT 0...",Cayman Islands dollar,CLFXGBTQ
9,CLFXGBTQ,Bahraini dinar,Melanie,Murray,"4561 Julie Shoal\nWilsonmouth, RI 68826",Cayman Islands dollar,CLFXGBTQ


,bank_id,bank_default_currency,first_name,last_name,address,default_currency,bank_id
90,ZWQRGB760TN,Cayman Islands dollar,Carly,Johnson,"82174 Eric Islands\nMelissahaven, OK 09352",Burmese kyat,ZWQRGB760TN
91,ZWQRGB760TN,Cayman Islands dollar,Janet,Navarro,"70223 Allen Causeway\nLake Kenneth, WV 13881",Sudanese pound,ZWQRGB760TN
92,ZWQRGB760TN,Cayman Islands dollar,Jonathon,Obrien,"820 Johnson Estates\nNorth Shawn, IN 67119",Australian dollar,ZWQRGB760TN
93,ZWQRGB760TN,Cayman Islands dollar,Matthew,Kim,"5000 Jamie Centers Suite 833\nPort Chase, AK 1...",Sudanese pound,ZWQRGB760TN
94,ZWQRGB760TN,Cayman Islands dollar,Patricia,Martin,"81523 Ryan Bypass Suite 843\nLake Raymond, IN ...",Burmese kyat,ZWQRGB760TN
95,ZWQRGB760TN,Cayman Islands dollar,Peter,Mitchell,"5371 Kristen Isle\nDonaldstad, KS 12184",Burmese kyat,ZWQRGB760TN
96,ZWQRGB760TN,Cayman Islands dollar,Philip,Smith,30325 Hudson Throughway Suite 692\nNorth Curti...,Cayman Islands dollar,ZWQRGB760TN
97,ZWQRGB760TN,Cayman Islands dollar,Richard,Blankenship,"38310 Miller Walk\nBrandibury, AR 69388",Burmese kyat,ZWQRGB760TN
98,ZWQRGB760TN,Cayman Islands dollar,Sabrina,Erickson,"71419 Claudia Union\nJonathanchester, KY 69469",Bahraini dinar,ZWQRGB760TN
99,ZWQRGB760TN,Cayman Islands dollar,Shawn,Richmond,Unit 6795 Box 8522\nDPO AE 87607,Hungarian forint,ZWQRGB760TN


In [13]:
# When reviewing the preceding image, remember that the join returns all the columns from both tables - but only for the rows where both tables have matching values in the 'bank_id' column.
# In this case, every row in the `customers` table contains a 'bank_id' value that the `banks` table also contains.
# So, the join returns a DataFrame that contains the full 100 rows. 
# Furthermore, the DataFrame contains seven columns: Two from the `banks` table, and five from the `customers` table.
# By default, the `JOIN` clause performs an inner join on the data.
# But, we can specify the type of join to use. 
# For example, the following code specifies the creation of a left join:
query = """
SELECT *
FROM banks
LEFT JOIN customers ON banks.bank_id = customers.bank_id;
"""
left_join_df = pd.read_sql_query(query, con=engine)
display(left_join_df.head(10))
display(left_join_df.tail(10))

,bank_id,bank_default_currency,first_name,last_name,address,default_currency,bank_id
0,CLFXGBTQ,Bahraini dinar,Aaron,Norton,"24449 Tammy Via Suite 496\nWebbfurt, WI 39109",Cayman Islands dollar,CLFXGBTQ
1,CLFXGBTQ,Bahraini dinar,Alexandria,Ramirez,Unit 5133 Box 6283\nDPO AP 51610,Bahraini dinar,CLFXGBTQ
2,CLFXGBTQ,Bahraini dinar,Andre,Fowler,"76574 Brown Road\nLake Danny, GU 12296",Central African CFA franc,CLFXGBTQ
3,CLFXGBTQ,Bahraini dinar,Brandi,Johnson,"30482 Tyler Keys\nLake Ryanside, RI 21937",Zimbabwean dollar,CLFXGBTQ
4,CLFXGBTQ,Bahraini dinar,Brenda,Marshall,"570 Tonya Walk\nCynthiaville, PR 25458",Australian dollar,CLFXGBTQ
5,CLFXGBTQ,Bahraini dinar,Curtis,Roberts,"0337 Hayes Trace\nNorth Kylehaven, RI 85697",Hungarian forint,CLFXGBTQ
6,CLFXGBTQ,Bahraini dinar,Matthew,Le,"358 Christina Alley Apt. 666\nBrennanview, MT ...",Sudanese pound,CLFXGBTQ
7,CLFXGBTQ,Bahraini dinar,Matthew,Norman,"4183 Allen Vista Suite 238\nEast Brenda, IA 52658",Australian dollar,CLFXGBTQ
8,CLFXGBTQ,Bahraini dinar,Melanie,Carrillo,"04410 Sarah Harbor\nSouth Elizabethburgh, MT 0...",Cayman Islands dollar,CLFXGBTQ
9,CLFXGBTQ,Bahraini dinar,Melanie,Murray,"4561 Julie Shoal\nWilsonmouth, RI 68826",Cayman Islands dollar,CLFXGBTQ


,bank_id,bank_default_currency,first_name,last_name,address,default_currency,bank_id
90,ZWQRGB760TN,Cayman Islands dollar,Carly,Johnson,"82174 Eric Islands\nMelissahaven, OK 09352",Burmese kyat,ZWQRGB760TN
91,ZWQRGB760TN,Cayman Islands dollar,Janet,Navarro,"70223 Allen Causeway\nLake Kenneth, WV 13881",Sudanese pound,ZWQRGB760TN
92,ZWQRGB760TN,Cayman Islands dollar,Jonathon,Obrien,"820 Johnson Estates\nNorth Shawn, IN 67119",Australian dollar,ZWQRGB760TN
93,ZWQRGB760TN,Cayman Islands dollar,Matthew,Kim,"5000 Jamie Centers Suite 833\nPort Chase, AK 1...",Sudanese pound,ZWQRGB760TN
94,ZWQRGB760TN,Cayman Islands dollar,Patricia,Martin,"81523 Ryan Bypass Suite 843\nLake Raymond, IN ...",Burmese kyat,ZWQRGB760TN
95,ZWQRGB760TN,Cayman Islands dollar,Peter,Mitchell,"5371 Kristen Isle\nDonaldstad, KS 12184",Burmese kyat,ZWQRGB760TN
96,ZWQRGB760TN,Cayman Islands dollar,Philip,Smith,30325 Hudson Throughway Suite 692\nNorth Curti...,Cayman Islands dollar,ZWQRGB760TN
97,ZWQRGB760TN,Cayman Islands dollar,Richard,Blankenship,"38310 Miller Walk\nBrandibury, AR 69388",Burmese kyat,ZWQRGB760TN
98,ZWQRGB760TN,Cayman Islands dollar,Sabrina,Erickson,"71419 Claudia Union\nJonathanchester, KY 69469",Bahraini dinar,ZWQRGB760TN
99,ZWQRGB760TN,Cayman Islands dollar,Shawn,Richmond,Unit 6795 Box 8522\nDPO AE 87607,Hungarian forint,ZWQRGB760TN


In [14]:
# Because both the `banks` and `customers` tables contain matching elements in the 'bank_id' column, the DataFrame resulting from the query consists of the combined data from both tables.
# Note that all seven columns are present in this new DataFrame.
# With `LEFT JOIN`, even if the `customers` table had 'bank_id' values that didn't exist in the `banks` table, we would still get a DataFrame consisting of all the customer data.
# Remember that a left join always returns all the rows of data in Table A.

# IMPORTANT:
# The SQLite database we're using doesn't currently support right joins or full outer joins.
# If you need that capability, you'll need to use a different database, such as PostgreSQL or MySQL.
# Remember that you can use almost any major database with Python and SQLAlchemy. 
# You can read more about the other supported database types on the SQLAlchemy Dialects Page: https://docs.sqlalchemy.org/en/13/dialects/index.html

In [18]:
# ANALYZE THE JOINED TABLES
# Because our left join resulted in a DataFrame that combined all the information from both tables, we can do various analysis tasks on it.
# For example, say that we want to find the data points where the bank's default currency is the same as the customer's default currency.
# We can find those matches by adding a conditional `WHERE` clause to our query:
query = """
SELECT * 
FROM banks
JOIN customers ON banks.bank_id = customers.bank_id
WHERE banks.bank_default_currency = customers.default_currency;
"""
join_where_df = pd.read_sql_query(query, con=engine)
display(join_where_df.head())
display(join_where_df.tail())

,bank_id,bank_default_currency,first_name,last_name,address,default_currency,bank_id
0,CLFXGBTQ,Bahraini dinar,Alexandria,Ramirez,Unit 5133 Box 6283\nDPO AP 51610,Bahraini dinar,CLFXGBTQ
1,CLFXGBTQ,Bahraini dinar,Rachael,Bell,"9821 Ashley Tunnel\nPort Matthew, CO 24979",Bahraini dinar,CLFXGBTQ
2,TBSVGBK1,Central African CFA franc,Eric,Weeks,"1518 Michael Manors\nWest Angela, AK 99107",Central African CFA franc,TBSVGBK1
3,ZXQLGB826RC,Zimbabwean dollar,Katie,Mclaughlin,"89923 Amanda Island\nNew John, TX 60181",Zimbabwean dollar,ZXQLGB826RC
4,ZXQLGB826RC,Zimbabwean dollar,Sierra,Robinson,"8282 Jensen Drive Apt. 820\nJessicaton, RI 98014",Zimbabwean dollar,ZXQLGB826RC


,bank_id,bank_default_currency,first_name,last_name,address,default_currency,bank_id
9,LOFYGBZ37P5,Venezuelan bolívar,Emily,Becker,"28615 Jennifer Glens Apt. 273\nBaxterstad, WY ...",Venezuelan bolívar,LOFYGBZ37P5
10,QQZLGB33W93,Japanese yen,Dominique,Johnson,"36902 Judy Squares\nFernandezbury, MP 19436",Japanese yen,QQZLGB33W93
11,KWFTGBR4,Sudanese pound,Amanda,Roth,"66638 Garcia Pass\nAprilfort, OR 09714",Sudanese pound,KWFTGBR4
12,KWFTGBR4,Sudanese pound,Jason,Kelly,"77472 Ewing Forge Suite 521\nSouth Jake, TX 49494",Sudanese pound,KWFTGBR4
13,ZWQRGB760TN,Cayman Islands dollar,Philip,Smith,30325 Hudson Throughway Suite 692\nNorth Curti...,Cayman Islands dollar,ZWQRGB760TN


In [ ]:
# The combination of the `JOIN` clause and the `WHERE` clause results in a smaller DataFrame.
# this new DataFrame has the same number of columns but only 14 rows of data rather than 100 rows.
# By combining joins with `SELECT` statements, we can create sophisticated database queries.